In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
from time_compressed_algo import register_ as reg
from time_compressed_algo import algo_cooling as algo
from time_compressed_algo import quf
import time
import numpy as np
from quimb.tensor.belief_propagation.l2bp import L2BP
from quimb.experimental import tnvmc

In [24]:
reg.reg_complex_svd()
import torch
to_backend = algo.backend_torch(device = "cpu", dtype = torch.float64, requires_grad=False)
to_backend_c = algo.backend_torch(device = "cpu", dtype = torch.complex128, requires_grad=False)

opt = algo.opt_(progbar=False)

In [3]:
#ITF params
J = 1
h = 3.05
delta = 0.05
Lx, Ly = 4, 4
L = Lx * Ly
edges = qtn.edges_2d_square(Lx=Lx, Ly=Ly, cyclic=False)

sites = sorted({ (site,) for edge in edges for site in edge})
N = len(sites)
N

16

In [4]:
pepo_t = quf.pepo_trotter_ITF(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=delta)
pepo_t = quf.pepo_trotter_ITF_fourth(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=delta)
pepo_t.max_bond()

8

In [5]:
chi = 3
pepo_raw = pepo_t.copy()


bp = L2BP(pepo_t, site_tags=list(pepo_t.site_tags), damping=0.0, optimize=opt, 
          normalize = 'L2phased', update='parallel',
         )
bp.run(tol=1.e-7, max_iterations=2000, progbar=True, diis = True)
pepo_t = bp.compress(pepo_t, max_bond=chi, cutoff=1.-12, cutoff_mode="rsum2", renorm=0, lazy=False)

mantissa, norm_exponent = bp.contract(strip_exponent=True)
est_norm = complex(mantissa * 10**norm_exponent)
print("bp_norm", est_norm.real, est_norm.real/2**L)

# print("check",  complex((pepo_raw.H & pepo_raw).contract(all, optimize=opt)).real, complex(norm).real)
# print("check_",  complex((pepo.H & pepo).contract(all, optimize=opt)).real)



chi_ = 64
copt = algo.copt_(progbar=True, chi=chi_, directory=None, max_repeats=2**9)
f = quf.loss_peps(pepo_t, pepo_raw, opt, copt, cost_f="fid", val_=2**L, chi_bmps=chi_, mode="exact", progbar=True)
#print("g")
#f_ = quf.loss_peps_g(pepo, pepo_raw, opt, copt, cost_f="fid", chi_bmps=chi_, mode = "exact", progbar=False)
print(complex(f).real)

max|dM|=7.25e-16 nconv: 48/48 : : 2it [00:01,  1.11it/s]


bp_norm 65536.00000000009 1.0000000000000013
1.108853142639532e-08


In [32]:
pepo_t = quf.pepo_trotter_ITF(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=delta)
pepo_t = quf.pepo_trotter_ITF_fourth(edges, Lx=Lx, Ly=Ly, to_backend=to_backend_c, h=h, J=J, delta=delta)
pepo_t.max_bond()

8

In [33]:
pepo_raw = pepo_t.copy()
chi = 2
# bp = L2BP(pepo_t, site_tags=list(pepo_t.site_tags), damping=0.0, optimize=opt, 
#           normalize = 'L2phased', update='parallel',
#          )
# bp.run(tol=1.e-7, max_iterations=2000, progbar=True, diis = True)
# pepo_t = bp.compress(pepo_t, max_bond=chi, cutoff=1.-12, cutoff_mode="rsum2", renorm=0, lazy=False)
# mantissa, norm_exponent = bp.contract(strip_exponent=True)
# est_norm = complex(mantissa * 10**norm_exponent)
# print("bp_norm", est_norm.real, est_norm.real/2**L)


pepo_t = qtn.PEPO.rand(Lx=Lx, Ly=Ly, bond_dim=chi, seed=666, dtype="complex128")
pepo_t.apply_to_arrays(to_backend_c)

print( (pepo_t.H & pepo_t).contract(all, optimize=opt)  )

chi_ = 64
copt = algo.copt_(progbar=True, chi=chi_, directory=None, max_repeats=2**9)
#f = quf.loss_peps(pepo_t, pepo_raw, opt, copt, cost_f="fid", val_=2**L, chi_bmps=chi_, mode="exact", progbar=True)
#print("g")
f_ = quf.loss_peps_g(pepo_t, pepo_raw, opt, copt, cost_f="fid", chi_bmps=chi_, mode = "exact", progbar=False)
print(complex(f_).real)

chi_ = 64
copt = algo.copt_(progbar=True, chi=chi_, directory=None, max_repeats=2**9)



tnopt = qtn.TNOptimizer(
    pepo_t,                       
    quf.loss_peps,
    loss_constants = {"peps_fix":pepo_raw},
    loss_kwargs = {'opt': opt, 'cost_f': "fid", "val_": (2.)**(L), "chi_bmps":chi, "mode": "exact", "copt":copt},
    autodiff_backend = "torch",
    optimizer= "L-BFGS-B", #'L-BFGS-B',
    progbar=True, jit_fn=False, device="cpu",
    #shared_tags=tags_opt,
)






tensor(2.0954e+20+320.3833j, dtype=torch.complex128)
-41828548228713.99


In [35]:
pepo_t = tnopt.optimize(n=9220, tol=2.220e-12, maxfun=10e+9, gtol= 1e-12, eps=1.49016e-09, maxls=400)


+0.000003462414 [best: +0.000003462414] :   0%|                                                     | 3/9220 [00:01<1:34:14,  1.63it/s]


In [20]:
pepo = qtn.PEPO.rand(Lx=Lx, Ly=Ly, bond_dim=4, seed=666, dtype="complex128")
